# Investigating correlation between features and tumor stage

In [6]:
setwd("/home/data/project_code/landstrom_core/prognostic_model_development/r/notebooks")
library(ggplot2)
library(tidyverse)
source("../getTCGAData.R")
source("../preprocessTCGAData.R")
source("../KM_analysis.R")
source("../Heatmaps.R")
source("../enet.R")
library(ggpubr)
library(rstatix)

── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1
✔ purrr   0.3.4     

── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call,

# Function

In [7]:
#
# Define function for adding the clinical variables 
#
addClinVar = function(data, clin.var) {
    if ("Age.oth" %in% clin.var) {
        data$Age.oth <- data$age_at_diagnosis.clin
    } 
    if ("Tumor.stage.oth" %in% clin.var){
        data$Tumor.stage.oth = factor(map_chr(data$ajcc_pathologic_stage.clin, reformatTumorStage))
    }
    if ("Gender.oth" %in% clin.var){
        data$Gender.oth <- factor(data$gender.clin)    
    } 
    if ("Gleason.group.oth" %in% clin.var) {
        
        # Determine the Gleason group 
        data$Gleason.group.oth = map2_chr(data$primary_gleason_grade.clin, 
                                           data$secondary_gleason_grade.clin, 
                                           determineGleasonGroup)

        # Set up the factor levels 
        data$Gleason.group.oth = factor(data$Gleason.group, 
                                    levels = c("Gleason group 1", "Gleason group 2"))
    }
    return(data)
}

In [64]:
# Helper 
incModel = function(x, models){
    res = c()
    x = paste(x, "expression")
    print(models %>% select(End.point, Feature))
    for (i in 1:nrow(models)){
        # Get model features as vector
        model.feat = unlist(str_split(models$Feature[i], ","))
        model.clin.end = models$End.point[i]
        print(model.clin.end)
        if (x %in% model.feat){
            res = c(res, model.clin.end)
        }
    }
    if (is.null(res) == F){
        return(paste(res, collapse = ","))
    }
    else 
        return("")
}

# Check features included
checkFeaturesIncluded = function(stats, eval.results){
    clin.models.inc.feat = unlist(map(stats$Feature, incModel, models = eval.results))
    stats$Included.in.models = clin.models.inc.feat
    return(stats)
}

# Setting up paths and variables

In [41]:
# Read in the table including the clinical features for each cancer type
clin.feat.tb = read.table("/workstation/project_data/landstrom_core/clin_features_final.csv", sep = "\t", header = T)

# Ensembl id mapping file 
ens.id.mapping = "/home/organisms/Human/hg38/Homo_sapiens.GRCh38_March2022/ENSEMBLE_to_SYMBOL.csv"

# Input dir data 
input.dir.data.path = file.path("/workstation/project_data/landstrom_core/rdata/manuscript_work/")

# Output directory
dir.res.root = file.path("/workstation/project_results/landstrom_core/Features_vs_tumor_stage/")
dir.create(dir.res.root)

# Gene list  
gene.list.file = read.table("/workstation/project_data/landstrom_core/Customer_genes.tsv", 
                            sep = "\t", header = F)
gene.list = gene.list.file$V1

# Read in the prognostic model results 
eval.results.with.features = read.csv("/workstation/project_results/landstrom_core/prognostic_model_development/Manuscript_tables/Table_evaluation_best_clin_end_with_reference_features_inc.csv")

Warning message in dir.create(dir.res.root):
“'/workstation/project_results/landstrom_core/Features_vs_tumor_stage' already exists”


# Prepare boxplots and test for significance

Output dir :

In [42]:
# Output dir :
boxplt.results.dir = file.path(dir.res.root, "Features_and_tumor_stage_boxplots")
dir.create(boxplt.results.dir)

Warning message in dir.create(boxplt.results.dir):
“'/workstation/project_results/landstrom_core/Features_vs_tumor_stage//Features_and_tumor_stage_boxplots' already exists”


In [43]:
# Store all statistical test results to a table 
stat.results.ls = list()

In [44]:
eval.results.with.features$Cancer.type

[1] "ACC"           "ACC"           "ACC"           "BLCA"         
 [5] "BLCA"          "BLCA"          "BRCA"          "BRCA"         
 [9] "BRCA"          "BRCA"          "CESC"          "CESC"         
[13] "CESC"          "CESC"          "CHOL"          "CHOL"         
[17] "COAD"          "COAD"          "COAD"          "DLBC"         
[21] "DLBC"          "ESCA"          "ESCA"          "ESCA"         
[25] "GBM"           "GBM"           "HNSC"          "HNSC"         
[29] "HNSC"          "KICH"          "KICH"          "KICH"         
[33] "KIRC"          "KIRC"          "KIRC"          "KIRC"         
[37] "KIRP"          "KIRP"          "KIRP"          "KIRP"         
[41] "LAML"          "LGG"           "LGG"           "LGG"          
[45] "LGG"           "LIHC"          "LIHC"          "LIHC"         
[49] "LIHC"          "LUAD"          "LUAD"          "LUAD"         
[53] "LUAD"          "LUSC"          "LUSC"          "MESO"         
[57] "MESO"          "MESO"          "PAAD"          "PAAD"         
[61] "PAAD"          "PAAD"          "PRAD"          "PRAD"         
[65] "PRAD"          "READ_and_COAD" "READ_and_COAD" "READ_and_COAD"
[69] "SARC"          "SARC"          "SKCM"          "STAD"         
[73] "STAD"          "STAD"          "STAD"          "TGCT"         
[77] "THCA"          "THCA"          "THYM"          "UCEC"         
[81] "UCEC"          "UCEC"          "UVM"           "UVM"          
[85] "UVM"

Run analysis

In [65]:
plots.ls = list()
i = 1

for (cancer.type in clin.feat.tb$Ctype){
    
    # Get Clinical variables
    clin.var = unlist(strsplit(clin.feat.tb$Features[clin.feat.tb$Ctype == cancer.type], split = ","))
    
    # Add oth-suffix 
    clin.var = paste0(clin.var, ".oth")
    
    # Read in the preprocessed dataset if continued 
    tcga.dataset = readRDS(file.path(input.dir.data.path, cancer.type, "tcga.dataset.rds"))
    
    # Add clinical variables to dataset
    tcga.dataset = addClinVar(tcga.dataset, clin.var)
    
    # Selected variables 
    variables.selected = c(paste0(gene.list, ".exp"), "Tumor.stage.oth")
    
    # Selected variables         
    data.selected.exp = tcga.dataset %>% 
                        dplyr::select(one_of(variables.selected))
    
    if ("Tumor.stage.oth" %in% colnames(data.selected.exp)){
        
        data.selected.exp.long = tidyr::pivot_longer(data.selected.exp, -Tumor.stage.oth,
                                                         names_to = "Feature", 
                                                         values_to = "Expression")
        
        data.selected.exp.long = data.selected.exp.long %>% filter(is.na(Tumor.stage.oth) == F)
        
        # Fix feature names 
        data.selected.exp.long$Feature = stringr::str_replace(data.selected.exp.long$Feature,".exp","")

        # Calculate the max expression for each feature 
        y.max = data.selected.exp.long %>% 
             group_by(Feature) %>% 
             summarise(ymax = max(Expression, na.rm = T))

        y.max$ymax = y.max$ymax + 0.2

        stat.test <- data.selected.exp.long %>%
                  group_by(Feature) %>%
                  kruskal_test(Expression ~ Tumor.stage.oth)

        stat.test = dplyr::left_join(stat.test, y.max, by = "Feature")
        
        # Add 
        eval.results.cancer.type = eval.results.with.features %>% 
                                                filter(Cancer.type == cancer.type)
        
        # Check if features are included in the models
        # Returns stats.test with added column indicating in which models by Clin end point 
        # the feature occurs
        stat.test = checkFeaturesIncluded(stat.test, eval.results.cancer.type)
        print(stat.test)
        
        # Store result df to list
        stat.results.ls[[cancer.type]] = stat.test
        
        # Store the data
        plot.data = list(stat = stat.test, 
                         exp.data = data.selected.exp.long)
        saveRDS(object = plot.data, file = file.path(boxplt.results.dir, paste(cancer.type, sep = "_", "data_for_plot", ".rds")))
        
        # Prepare boxplot
        gg = data.selected.exp.long %>% 
               ggplot(aes(x = Feature)) + geom_boxplot(aes(y = Expression, fill = Tumor.stage.oth))+
        geom_text(data = stat.test, aes(x = Feature, label = paste0("p = ", p), y = ymax), position = position_dodge(0.8)) + 
        theme_pubr() + scale_fill_manual(values = c("#c6dbef", "#6baed6", "#2171b5", "#08306b"))


        plots.ls[[i]] = gg 
        
        ggsave(plot = gg,
              filename = file.path(boxplt.results.dir, paste0(cancer.type, "_boxplot.pdf")),
               height = 12, width = 16)
        
        i = i + 1
    
    }
}

Warning message:
“Unknown columns: `Tumor.stage.oth`”


  End.point                                  Feature
1        OS Age,Gender,Tumor stage,TGFBR1 expression
2       DSS        Age,Tumor stage,TGFBR1 expression
3       PFI            Tumor stage,TGFBR1 expression
[1] "OS"
[1] "DSS"
[1] "PFI"
  End.point                                  Feature
1        OS Age,Gender,Tumor stage,TGFBR1 expression
2       DSS        Age,Tumor stage,TGFBR1 expression
3       PFI            Tumor stage,TGFBR1 expression
[1] "OS"
[1] "DSS"
[1] "PFI"
  End.point                                  Feature
1        OS Age,Gender,Tumor stage,TGFBR1 expression
2       DSS        Age,Tumor stage,TGFBR1 expression
3       PFI            Tumor stage,TGFBR1 expression
[1] "OS"
[1] "DSS"
[1] "PFI"
  End.point                                  Feature
1        OS Age,Gender,Tumor stage,TGFBR1 expression
2       DSS        Age,Tumor stage,TGFBR1 expression
3       PFI            Tumor stage,TGFBR1 expression
[1] "OS"
[1] "DSS"
[1] "PFI"
  End.point                         

Warning message:
“Removed 52 rows containing non-finite values (stat_boxplot).”


  End.point                                  Feature
1        OS Age,Gender,Tumor stage,MYC amplification
2       DSS         Age,Tumor stage,AURKB expression
3       DFI  Age,Gender,Tumor stage,APPL2 expression
4       PFI             Tumor stage,KIF23 expression
[1] "OS"
[1] "DSS"
[1] "DFI"
[1] "PFI"
  End.point                                  Feature
1        OS Age,Gender,Tumor stage,MYC amplification
2       DSS         Age,Tumor stage,AURKB expression
3       DFI  Age,Gender,Tumor stage,APPL2 expression
4       PFI             Tumor stage,KIF23 expression
[1] "OS"
[1] "DSS"
[1] "DFI"
[1] "PFI"
  End.point                                  Feature
1        OS Age,Gender,Tumor stage,MYC amplification
2       DSS         Age,Tumor stage,AURKB expression
3       DFI  Age,Gender,Tumor stage,APPL2 expression
4       PFI             Tumor stage,KIF23 expression
[1] "OS"
[1] "DSS"
[1] "DFI"
[1] "PFI"
  End.point                                  Feature
1        OS Age,Gender,Tumor stage,

Warning message:
“Removed 169 rows containing non-finite values (stat_boxplot).”
Warning message:
“Unknown columns: `Tumor.stage.oth`”


...

  End.point
1        OS
2       DSS
3       DFI
4       PFI
                                                                                Feature
1                                Age,Gender,Tumor stage,TGFBR1 expression,AR expression
2 Age,Gender,Tumor stage,TP53 expression,AR expression,TGFBR1 expression,MYC expression
3                                  Gender,Tumor stage,AURKA expression,KIF23 expression
4       Gender,Tumor stage,MYC deletion,MYC amplification,TP53 expression,AR expression
[1] "OS"
[1] "DSS"
[1] "DFI"
[1] "PFI"
  End.point
1        OS
2       DSS
3       DFI
4       PFI
                                                                                Feature
1                                Age,Gender,Tumor stage,TGFBR1 expression,AR expression
2 Age,Gender,Tumor stage,TP53 expression,AR expression,TGFBR1 expression,MYC expression
3                                  Gender,Tumor stage,AURKA expression,KIF23 expression
4       Gender,Tumor stage,MYC deletion,MYC ampli

Warning message:
“Removed 832 rows containing non-finite values (stat_boxplot).”


  End.point                       Feature
1       DFI Age,Tumor stage,AR expression
[1] "DFI"
  End.point                       Feature
1       DFI Age,Tumor stage,AR expression
[1] "DFI"
  End.point                       Feature
1       DFI Age,Tumor stage,AR expression
[1] "DFI"
  End.point                       Feature
1       DFI Age,Tumor stage,AR expression
[1] "DFI"
  End.point                       Feature
1       DFI Age,Tumor stage,AR expression
[1] "DFI"
  End.point                       Feature
1       DFI Age,Tumor stage,AR expression
[1] "DFI"
  End.point                       Feature
1       DFI Age,Tumor stage,AR expression
[1] "DFI"
  End.point                       Feature
1       DFI Age,Tumor stage,AR expression
[1] "DFI"
  End.point                       Feature
1       DFI Age,Tumor stage,AR expression
[1] "DFI"
  End.point                       Feature
1       DFI Age,Tumor stage,AR expression
[1] "DFI"
  End.point                       Feature
1       DFI Age,Tu

Warning message:
“Removed 65 rows containing non-finite values (stat_boxplot).”
Warning message:
“Unknown columns: `Tumor.stage.oth`”
Warning message:
“Unknown columns: `Tumor.stage.oth`”


  End.point
1        OS
2       DSS
3       PFI
                                                                                Feature
1                          Tumor stage,APPL2 expression,MYC expression,AURKA expression
2 Age,Gender,Tumor stage,APPL2 expression,MYC expression,AURKA expression,AR expression
3                                         Tumor stage,AURKA expression,APPL1 expression
[1] "OS"
[1] "DSS"
[1] "PFI"
  End.point
1        OS
2       DSS
3       PFI
                                                                                Feature
1                          Tumor stage,APPL2 expression,MYC expression,AURKA expression
2 Age,Gender,Tumor stage,APPL2 expression,MYC expression,AURKA expression,AR expression
3                                         Tumor stage,AURKA expression,APPL1 expression
[1] "OS"
[1] "DSS"
[1] "PFI"
  End.point
1        OS
2       DSS
3       PFI
                                                                                Feature
1     

In [66]:
stat.results = bind_rows(stat.results.ls, .id = "Cancer type")
write.csv(stat.results, file.path(dir.res.root, "Features_and_tumor_stage_boxplots", "Features_vs_tumor_stage_stat_results.csv"))